In [3]:
from google.colab import drive
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

drive.mount('/content/drive', force_remount=True)
df = pd.read_excel("/content/drive/MyDrive/AI/dataset/Energy_Power_Weather.xlsx", sheet_name=0)

Mounted at /content/drive


# 1.Tiền xử lý dữ liệu

In [4]:
# df = pd.read_excel("Energy_Power_Weather.xlsx", sheet_name=0)
df = df.drop(labels=["Timestamp", "Date"], axis=1)
df

,kilowatt-hours,TempMin,TempMax,Weather
0,2.165,18,23,Patchy rain possible
1,6.470,20,25,Patchy rain possible
2,9.157,21,24,Patchy rain possible
3,16.269,21,25,Sunny
4,8.447,19,23,Moderate rain at times
...,...,...,...,...
1757,4.514,21,23,Light rain shower
1758,11.096,22,26,Patchy rain possible
1759,4.860,20,25,Light rain shower
1760,13.984,23,25,Light rain shower


In [5]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df["Weather"] = le.fit_transform(df["Weather"])
print(df["Weather"].unique())

[10 11  5  6  7  0  1  4  2  3  9 12  8 13]


# 2.Tạo dữ liệu

In [6]:
from sklearn.model_selection import train_test_split


X = []
y = []
dataframe = df.values
for i in range(len(dataframe) - 15):
    X.append(dataframe[i:i+14])
    y.append(dataframe[i+15][0])
X = np.array(X)
y = np.array(y)

print(X.shape, y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)

(1747, 14, 4) (1747,)
(1397, 14, 4)


# 3.Huấn luyện

In [7]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# Hàm đánh giá theo rmse

def evaluate_forecasts(actual, predicted):
	scores = list()
	# calculate an RMSE score for each day
	for i in range(actual.shape[1]):
		# calculate mse
		mse = mean_squared_error(actual[:, i], predicted[:, i])
		# calculate rmse
		rmse = sqrt(mse)
		# store
		scores.append(rmse)
	# calculate overall RMSE
	s = 0
	for row in range(actual.shape[0]):
		for col in range(actual.shape[1]):
			s += (actual[row, col] - predicted[row, col])**2
	score = sqrt(s / (actual.shape[0] * actual.shape[1]))
	return score, scores

### 3.1. LSTM

In [15]:
from keras import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.metrics import RootMeanSquaredError


def build_lstm(X_train, y_train):
    n_samples, timesteps, n_feature = X_train.shape
    model = Sequential()
    model.add(LSTM(200, activation="tanh", input_shape=(timesteps, n_feature)))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(1))
    model.compile(loss="mse", optimizer='adam', metrics=[RootMeanSquaredError()])
    model.fit(X_train, y_train, epochs=100, batch_size=16, verbose=0)
    return model

md = build_lstm(X_train, y_train)


In [16]:
print(md.evaluate(X_test, y_test, verbose=0))
print(md.predict(X_test, verbose=0)[:7], y_test[:7])

[28.587759017944336, 5.346752166748047]
[[13.776164]
 [ 7.150707]
 [ 9.004036]
 [14.474032]
 [ 4.743385]
 [13.875017]
 [10.250835]] [17.108  1.076  8.959 19.032 10.294 13.739  6.464]


### 3.2 CNN-LSTM

In [ ]:
from keras.layers import Conv2D
from keras.layers import Dense
from keras import Model
from keras.layers import InputLayer

def build_cnn_lstm(X_train, y_train):
    x = InputLayer()


